# Importing libraries which we are going to use

In [5]:
import numpy as np
import pandas as pd
import pickle
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.models import load_model
from keras.preprocessing import image
from pathlib import Path
from sklearn import metrics

In [6]:
train_path = '../../../Desktop/Apnea_Train'
test_path = '../../../Desktop/Apnea_Test'

In [7]:
train_batches = ImageDataGenerator().flow_from_directory(
    train_path, class_mode='categorical', 
    target_size = (40, 300), batch_size = 32)

Found 26772 images belonging to 2 classes.


In [8]:
base_model = keras.applications.resnet50.ResNet50(
    weights = 'imagenet',include_top = False,input_shape = (40, 300, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(100,activation = 'relu'))
model.add(Dense(2,activation = 'softmax'))
# print(model.summary())

C:\Users\joey3\Anaconda3\envs\tf-gpu\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
model.compile(
    optimizer=Adam(), loss='categorical_crossentropy', 
    metrics=['accuracy'])

In [10]:
history = model.fit_generator(
    train_batches, epochs=5, steps_per_epoch = 836)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
836/836 [==============================] - 54s 64ms/step - loss: 0.3701 - acc: 0.8430
Epoch 2/5
836/836 [==============================] - 50s 60ms/step - loss: 0.3345 - acc: 0.8577
Epoch 3/5
836/836 [==============================] - 50s 60ms/step - loss: 0.3252 - acc: 0.8623
Epoch 4/5
836/836 [==============================] - 50s 59ms/step - loss: 0.3105 - acc: 0.8715
Epoch 5/5
836/836 [==============================] - 50s 60ms/step - loss: 0.3026 - acc: 0.8736


In [7]:
model.save('model_02011651.h5')

In [28]:
def predict_images():
#     test_images = []
#     for image_path in path_list:
#         test_image = image.load_img(image_path, target_size=(40, 300))
#         test_image = image.img_to_array(test_image)
#         test_image = np.expand_dims(test_image, axis=0)
#         test_images.append(test_image)

#     test_images = np.vstack(test_images)
#     y_pred = model.predict_classes(test_images, batch_size=8)
    
    y_pred, y_true = [], []
    for item in train_batches:
        temp = item[0]
        
        y_true += list(np.argmax(item[1], axis=1))
        y_pred += list(model.predict_classes(item[0]))
        print(len(y_true))
        if(len(y_true) > 26772):
            break
    
    return y_pred, y_true

In [22]:
for item in train_batches:
    temp = item[0]
    label = item[1]
    break

In [29]:
# file=train_path
# neg_list = Path(f'{file}').glob('0/*.png')
# pos_list = Path(f'{file}').glob('1/*.png')
K.set_learning_phase(1)
y_pred, y_true = predict_images()


print(metrics.classification_report(y_true, y_pred))

32
64
96
128
160
192
224
256
288
320
352
384
416
448
480
512
544
576
608
640
672
704
736
768
800
832
864
896
928
960
992
1024
1056
1088
1120
1152
1184
1216
1248
1280
1312
1344
1376
1408
1440
1472
1504
1536
1568
1600
1632
1664
1696
1728
1760
1792
1824
1856
1888
1920
1952
1984
2016
2048
2080
2112
2144
2176
2208
2240
2272
2304
2336
2368
2400
2432
2464
2496
2528
2560
2592
2624
2656
2688
2720
2752
2784
2816
2848
2880
2912
2944
2976
3008
3040
3072
3104
3136
3168
3200
3232
3264
3296
3328
3360
3392
3424
3456
3488
3520
3552
3584
3616
3648
3680
3712
3744
3776
3808
3840
3872
3904
3936
3968
4000
4032
4064
4096
4128
4160
4192
4224
4256
4288
4320
4352
4384
4416
4448
4480
4512
4544
4576
4608
4640
4672
4704
4736
4768
4800
4832
4864
4896
4928
4960
4992
5024
5056
5088
5120
5152
5184
5216
5248
5280
5312
5344
5376
5408
5440
5472
5504
5536
5568
5600
5632
5664
5696
5728
5760
5792
5824
5856
5888
5920
5952
5984
6016
6048
6080
6112
6144
6176
6208
6240
6272
6304
6336
6368
6400
6432
6464
6496
6528
6560
6592
6624

In [22]:
base_model.predict(temp).shape

(32, 2, 10, 2048)

In [33]:
y_pred = np.vstack([pos_pred, neg_pred])

pos_true, neg_true = np.ones(pos_pred.shape), np.zeros(neg_pred.shape)
y_true = np.vstack([pos_true, neg_true])

print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.89      0.52      0.66     16181
         1.0       0.55      0.90      0.68     10591

    accuracy                           0.67     26772
   macro avg       0.72      0.71      0.67     26772
weighted avg       0.76      0.67      0.67     26772



In [33]:
# test_df = pd.read_csv('resources/File_test.csv')
# for file in test_df['file']:
file=train_path
neg_list = Path(f'{file}').glob('0/*.png')
pos_list = Path(f'{file}').glob('1/*.png')

neg_pred = predict_images(neg_list)
pos_pred = predict_images(pos_list)
y_pred = np.vstack([pos_pred, neg_pred])

pos_true, neg_true = np.ones(pos_pred.shape), np.zeros(neg_pred.shape)
y_true = np.vstack([pos_true, neg_true])

print(metrics.classification_report(y_true, y_pred))

              precision    recall  f1-score   support

         0.0       0.89      0.52      0.66     16181
         1.0       0.55      0.90      0.68     10591

    accuracy                           0.67     26772
   macro avg       0.72      0.71      0.67     26772
weighted avg       0.76      0.67      0.67     26772

